# Loan Safety with Decision Trees

The LendingClub is a peer-to-peer lending company that directly connects borrowers and potential lenders/investors. In this notebook, you will build a classification model to predict whether or not a loan provided by LendingClub is likely to default. In this assignment, you will practice:

* Use Pandas Dataframes to do feature engineering
* Train a decision tree model to predict the sentiment of product reviews.
* Visualize the decision tree
* Predict the probability of a certain label using the tree
* Investigate how the complexity of the tree affects the results

Fill in the cells provided marked `TODO` with code to answer the questions. **Unless otherwise noted, every answer you submit should have code that clearly shows the answer in the output.** Answers submitted that do not have associated code that shows the answer may not be accepted for credit. 

**Make sure to restart the kernel and run all cells** (especially before turning it in) to make sure your code runs correctly. 

> Copyright ©2023 Emily Fox, Hunter Schafer, and Valentina Staneva  All rights reserved.  Permission is hereby granted to students registered for University of Washington CSE/STAT 416 for use solely during Spring Quarter 2024 for purposes of the course.  No other use, copying, distribution, or modification is permitted without prior written consent. Copyrights for third-party components of this work must be honored.  Instructors interested in reusing these course materials should contact the author.

---

In [166]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
sns.set()
np.random.seed(416)

# Investigate Data
In this first part of the assignment, we will investigate the data to get a better sense of what we are working with. 

> Remember that you should look through and understand the parts of the code that load in data since the later parts will assume you are familiar with the preprocessing we did.

First we load in the dataset and then inspect the values:

**NOTE**: Be sure to run every cell in the notebook! The `###SKIP` is for the autograder. 

In [1]:
### SKIP

# Set seed for the whole program
np.random.seed(416)

# Load in data
loans = pd.read_csv('lending-club-data.csv')
loans.head()

Do not modify the below cell. It configures the autograder, which will award 0 points if it doesn't run.


In [0]:
### edTest(test_load_data) ###

One of the features we will use in this assignment is the "grade" of the loan. We can investigate what this feature, "grade", looks like:

In [168]:
# Want the grades to show up in order from high to low
grade_order = sorted(loans['grade'].unique())

sns.countplot(x='grade', data=loans, order=grade_order)

We can see that over half of the loan grades are assigned values A or B. Each loan is assigned one of these grades, along with a more finely discretized feature called subgrade (feel free to explore that feature column as well!). These values depend on the loan application and credit report, and determine the interest rate of the loan. More information if you're interested can be found [here](https://www.lendingclub.com/investing/investor-education/interest-rates-and-fees).

Now let's look at another feature that will be used, "home ownership". This feature describes whether the loanee is mortaging, renting, or owns a home. We can see that a small percentage of the loanees own a home.

In [169]:
ownership_order = sorted(loans['home_ownership'].unique())

sns.countplot(x='home_ownership', data=loans, order=ownership_order)

## Investigate Data - Target
The target column (label column) of the dataset that we are interested in is called `bad_loans`. In this column 1 means a risky (bad) loan 0 means a safe loan.

In order to make this more intuitive and consistent with the lectures, we reassign the target to be:

* +1 as a safe loan,
* -1 as a risky (bad) loan.

We put this in a new column called `safe_loans`. This column will be the target values you will use while training.

In [170]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)

# Drop the old bad_loans column
loans = loans.drop(columns='bad_loans')

Now, let's explore the distribution of values for `safe_loans`. This gives us a sense of how many safe and risky loans are present in the dataset. 

In [171]:
only_safe = loans[loans['safe_loans'] == 1]
only_bad = loans[loans['safe_loans'] == -1]

print(f'Number safe  loans: {len(only_safe)} ({len(only_safe) * 100.0 / len(loans):.2f}%)')
print(f'Number risky loans: {len(only_bad)} ({len(only_bad) * 100.0 / len(loans):.2f}%)')

## Investigate Data - Questions

### **🔍 Q1) Most frequent grade**
Write code to compute the most frequent value in the `'grade'` column of our dataset.  Store the grade as a `str` in the variable `mode_grade`. You may assume that there is no tie in the most frequent grade.

**Like all other problems, unless otherwise specified you need to write code to compute this value in the data rather than hard-coding the answer your find by some other means (e.g., looking at the plot above)**

In [172]:
### edTest(test_q1_most_common_loan_grade) ###

# Q1: Write code to find most frequent grade
# TODO 

mode_grade = None

### **🔍 Q2) Percentage Rent**
What percent of the loans in our dataset are for renting? Write code to compute this number as a value between 0 and 1, and store it in a variable named `percent_rent`.

In [173]:
### edTest(test_q2_percent_rent) ###

# Q2: Write code to find percent of loans for rent
# TODO 

percent_rent = None

---

# Preprocess Data for Learning
In this section, we pre-process the data to make it suitable for ML.

## Encode Categorical Features
We will be using both numeric and categorical features to predict if a loan is safe or risky. The features are described in code commments in the next cell. 

In [174]:
features = [
    'grade',                     # grade of the loan (e.g. A or B)
    'sub_grade',                 # sub-grade of the loan (e.g. A1, A2, B1)
    'short_emp',                 # one year or less of employment (0 or 1)
    'emp_length_num',            # number of years of employment (a number)
    'home_ownership',            # home_ownership status (one of own, mortgage, rent or other)
    'dti',                       # debt to income ratio (a number)
    'purpose',                   # the purpose of the loan (one of many values)
    'term',                      # the term of the loan (36 months or 60 months)
    'last_delinq_none',          # has borrower had a delinquincy (0 or 1)
    'last_major_derog_none',     # has borrower had 90 day or worse rating (0 or 1)
    'revol_util',                # percent of available credit being used (number between 0 and 100)
    'total_rec_late_fee',        # total late fees received to day (a number)
]

target = 'safe_loans'                   # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]
loans.head()

In [175]:
loans.columns

With how `sklearn` implemented its `DecisionTreeClassifier`, it is only able to handle numerical features; the technical reason being it only allows binary splits and assumes (like other `sklearn` models), that the data is only numerical. This means we need to translate the categorical features into numeric ones. 

The easiest way to do this is a **one-hot encoding** of each categorical feature. A one hot encoding of a feature creates new, derived, features that take on the value 0 or 1 (one new feature for each possible value of the original). This allows us to turn a categorical feature into a series of binary (0 or 1) numerical features.

For examples, if we had the dataset shown below where 'Gender' takes on values 'M' (Male), 'F' (Female), 'O' (Other) and 'HasHouse' takes on values 'Y' (Yes), and 'N' (No). This dataset would not work by default in scikit-learn since it has features with categorical values.

| Gender | HasHouse | Age |
|--------|----------|-----|
| M      | N        | 19  |
| F      | Y        | 23  |
| O      | Y        | 24  |
| F      | N        | 21  |

To fix this, we could use a one-hot encoding to transform the categorical features into numeric ones. A one-hot encoding of the categorical features would be 

| Gender_M | Gender_F | Gender_O | HasHouse_N | HasHouse_Y | Age |
|----------|----------|----------|------------|------------|-----|
| 1        | 0        | 0        | 1          | 0          | 19  |
| 0        | 1        | 0        | 0          | 1          | 23  |
| 0        | 0        | 1        | 0          | 1          | 24  |
| 0        | 1        | 0        | 1          | 0          | 21  |

Note that the original 'Gender' column has been transformed into three related columns 'Gender_M', 'Gender_F', and 'Gender_O' which contains zeros or ones depending on the rows' original value. For each row, only one of these derived columns will contain a one (hence the name one-hot encoding).

To do this in `pandas`, we use the `get_dummies()` method (another name for one-hot encoding, is dummy variable). We show the new features of the data after running the function in the next cell.

In [176]:
loans = pd.get_dummies(loans)
features = list(loans.columns)
features.remove('safe_loans')
features

Here is a preview of the new dataset. Notice our categorical features have become 0/1s represented over many columns (e.g. "grade" becamse "grade_A" through "grade_F").

In [177]:
loans.head()

---
# Model Training
In this section, you will start writing code to train the decision tree model. 

First we split the data into 80% training data and 20% validation data. For this assignment, we will not use a test set since we aren't going to be deploying this model in the future and don't need to make a claim on our accuracy guarantee. 

In [178]:
from sklearn.model_selection import train_test_split

train_data, validation_data = train_test_split(loans, test_size=0.2)

## **🔍 Q3) Train first classifier**

Now let's use `sklearn`'s `DecisionTreeClassifier` (documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)) 
to create a loan-safety prediction model on the training data. 

Train a decision tree classifier for this data. When creating the model you should only pass in the values `max_depth=6`. Use the features and target defined earlier when training the model on the training data. Save the tree in a variable called `decision_tree_model`.

In [202]:
### edTest(test_q3_decision_tree_model) ###

from sklearn.tree import DecisionTreeClassifier

# Q3: Train a model with max_depth=6
# TODO 
decision_tree_model = None


Next, we want you to visualize what the tree looks like. In the cell below, we have defined a function to visualize the tree. 

Visualizing a depth 7 tree can be hard, so instead we will train another model stored in a variable called `small_tree_model` that has `max_depth=2`.

Installing Graphviz (especially on Windows) can be kind of tricky. If the code below doesn't run correctly, we provide an image to the picture you should have seen! 

In [180]:
import graphviz
from sklearn import tree


def draw_tree(tree_model, features):
    """
    visualizes a Decision Tree
    """
    tree_data = tree.export_graphviz(tree_model, 
                                    impurity=False, 
                                    feature_names=features,
                                    class_names=tree_model.classes_.astype(str),
                                    filled=True,
                                    out_file=None)
    graph = graphviz.Source(tree_data) 
    display(graph)
    
small_tree_model = DecisionTreeClassifier(max_depth=2, random_state=0)
small_tree_model.fit(train_data[features], train_data[target])
draw_tree(small_tree_model, features)

Each node in the tree shows
* If it's an internal node, show what feature and value it splits on
* `samples`: The number of samples at that node
* `value`: The counts for each label at that node
* `class`: The majority class at that node
* The color showing how confident it is in the predictions at that node (blue is class +1, orange is class -1, white is in between). This corresponds to the probability of it predicting a certain label.

## Making Predictions
Now that we have trained the model, let's look at how it makes predictions on our data.

### **🔍 Q4) Accuracies**
Compute the training accuracy and validation accuracy using both your `decision_tree_model`. Calculate which percentage of those examples it classified correctly as a number between 0 and 1. So for this problem, you will need to make two variables:
* `decision_train_accuracy` for `decision_tree_model`'s training accuracy
* `decision_validation_accuracy` for `decision_tree_model`'s validation accuracy

In [181]:
### edTest(test_q4_validation_accuracy) ###

# Q4: Find train and validation accuracy
# TODO 

decision_train_accuracy = None
decision_validation_accuracy = None

##

### **🔍 Q5) Tall Tree**
Next, you should train another decision tree model with `max_depth=10`. This will make a much deeper tree. 

In the cell below, train the model on the training data and report its training and validation accuracy. Save the model in a variable called `big_tree_model`, the training accuracy in a variable called `big_train_accuracy` and validation accuracy `big_validation_accuracy`.

Look back at the values in the last question to see if your values here make sense with what you know about heights of trees.

In [182]:
### edTest(test_q5_big_tree) ###

# Q5: Train a decision tree model with max_depth=10
# TODO 

big_tree_model = None
big_train_accuracy = None
big_validation_accuracy = None


---
# **Q6) 🔍 Finding Right Fit**
As we saw in the previous example, if we aren't careful with how we set `max_depth`, our model can potentially overfit or underfit. 

A common way of limiting the depth of the tree is to consider two early stopping rules:
* When when the tree reaches a maximum depth (`max_depth`)
* When the leaf nodes have too few data samples in them (`min_samples_leaf`)

Instead of writing the standard hyper-parameter tuning loop you've written before (i.e. loop over possible choices of something like $\lambda$, train on the training data with that value of $\lambda$, and validate using a validation set), we will use a `sklearn` module that does this for us using k-fold cross validation.

The reason we want to use a library for this task is it gets more complicated when we want to find the best combination of both `max_depth` and `min_samples_leaf`. The code becomes quite tedious to write since your loop would need to try all pairs of values for `max_depth` and `min_samples_leaf` and have some way of storing the results to compare.

Instead, we use use `sklearn`'s `GridSearchCV` (documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)) to do all this book-keeping for us! You should look at the documentation on that page (there is a good example underneath the method reference) to solve the following step.

Use `GridSearchCV` to try all combinations of 
* `min_samples_leaf`: [1, 10, 50, 100, 200, 300]
* `max_depth`: [1, 5, 10, 15, 20]

Some implementation details:
* For our code later, save the `GridSearchCV` object in a variable called `search` and the dictionary specifying parameters in a variable called `hyperparameters`.
* You should use 6-fold valudation `cv=6` and make sure it records the training accuracies by using `return_train_score=True`.
* This might take some time to run! 

In [183]:
### edTest(test_q6_grid_search) ###

# Q6: Use GridSearchCV to find best settings of hyperparameters
# TODO 

search = None

Print the result using the `best_params_` property on the `GridSearchCV` object. 

In [0]:
print(search.best_params_)

We can also plot the train and validation accuracy of the models for different settings of the hyper-parameters. The plot will be in 3D since there are 2 inputs for each model specification.

In [184]:
def plot_scores(ax, title, search, hyperparameters, score_key):
    # Get results from GridSearch and turn scores into matrix
    cv_results = search.cv_results_
    scores = cv_results[score_key]
    scores = scores.reshape((len(hyperparameters['max_depth']), len(hyperparameters['min_samples_leaf'])))
    max_depths = cv_results['param_max_depth'].reshape(scores.shape).data.astype(int)
    min_samples_leafs = cv_results['param_min_samples_leaf'].reshape(scores.shape).data.astype(int)
    
    # Plot result
    ax.plot_wireframe(max_depths, min_samples_leafs, scores)
    ax.view_init(20, 220)
    ax.set_xlabel('Maximum Depth')
    ax.set_ylabel('Minimum Samples Leaf')
    ax.set_zlabel('Accuracy')
    ax.set_title(title)


fig = plt.figure(figsize=(15,7))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
plot_scores(ax1, 'Train Accuracy', search, hyperparameters, 'mean_train_score')
plot_scores(ax2, 'Validation Accuracy', search, hyperparameters, 'mean_test_score')

There is no question for this part, but make sure you understand why we see the trends we do here. Make sure you can answer the following questions:
* Why does the train accuracy spike up in the left graph, but go downward on the right graph?
* Which graph should we look at to pick the model that will hopefully do best in the future?
* Why is the spike in training accuracy only for the point with `max_depth=20` and `min_samples_leaf=1`? Why isn't the training accuracy very high for the other settings of `min_samples_leaf` even when `max_depth=20`?

--- 
# **🔍 Q7) Random Forest**
In class, we talked about the random forest ensemble. To show us empirically that such a simple idea of adding multiple classifiers works in practice, in this section we are going to implement a type of random forest and analyze its performance. 

This is slightly different than most work you'll do in our course since we are asking you to implement the inner-workings of part of a model. You should write code to make the ensemble, but you can use `sklearn`'s decision tree model as the models in your ensemble; in other words you don't need to implement a decision tree from scratch! 

Below, we have written the starter code for a class called `RandomForest416` that has methods similar to most sklearn models. You only have to implement the `fit` method, all other parts are implemented for you. However, you should understand how the other code works to help you identify what you need to do. Besides, this implement doesn't use parallelization like in practice, but technically each iteration is independent of the others.

The `fit` method should train each tree on a different random sample of the examples in the given dataset with $n$ examples. Each random sample should select examples uniformly at random (with replacement) to make a new, modified, dataset of $n$ examples for that tree. To do this, you will want to use `np.random.randint` (documentation [here](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html)) to generate a random sequence of indices, and then use the `iloc` property on the given `pandas` objects to select those rows. You do **not** need to randomly select the subset of features to use for each sample of the dataset; you can simply use all the features for each tree.

In [185]:
import scipy.stats 

class RandomForest416: 
    """
    This class implements the common sklearn model interface (has a fit and predict function).
    
    A random forest is a collection of decision trees that are trained on random subsets of the 
    dataset. When predicting the value for an example, takes a majority vote from the trees.
    """
    
    def __init__(self, num_trees, max_depth=1):
        """
        Constructs a RandomForest416 that uses the given numbner of trees, each with a 
        max depth of max_depth.
        """
        self._trees = [
            DecisionTreeClassifier(max_depth=max_depth) 
            for i in range(num_trees)
        ]
        
    def fit(self, X, y):
        """
        Takes an input dataset X and a series of targets y and trains the RandomForest416.
        
        Each tree will be trained on a random sample of the data that samples the examples
        uniformly at random (with replacement). Each random dataset will have the same number
        of examples as the original dataset, but some examples may be missing or appear more 
        than once due to the random sampling with replacement.
        """    
        # Q7
        # TODO 
           
    def predict(self, X):
        """
        Takes an input dataset X and returns the predictions for each example in X.
        """
        # Builds up a 2d array with n rows and T columns
        # where n is the number of points to classify and T is the number of trees
        predictions = np.zeros((len(X), len(self._trees)))
        for i, tree in enumerate(self._trees):
            # Make predictions using the current tree
            preds = tree.predict(X)
            
            # Store those predictions in ith column of the 2d array
            predictions[:, i] = preds
            
        # For each row of predictions, find the most frequent label (axis=1 means across columns)
return scipy.stats.mode(predictions, axis=1, keepdims=False).mode 

You can test out your implementation by creating RandomForest416 object using 2 decision trees, each with a max depth of 1. Save the training accuracy and validation accuracy of this model with `rf_train_accuracy` and `rf_validation_accuracy`

In [186]:
### edTest(test_q7_random_forest) ###

# Q7
# TODO

rf_train_accuracy = None
rf_validation_accuracy = None

We can then compare how `sklearn`'s `DecisionTreeClassifier` to our `RandomForest416` to see how its training and validation accuracies compare as a function of the depth of the trees. The code below trains each model with a different max depth and then plots their accuracies. It might take some time to run, but it also prints the progress by showing the current depth it is on (it will range from 1 to 25).

In [187]:
# First calculate the accuracies for each depth
depths = list(range(1, 26, 2))
dt_accuracies = []
rf_accuracies = []

for i in depths:
    print(f'Depth {i}')

    # Train and evaluate a Decision Tree Classifier with given max_depth
    tree = DecisionTreeClassifier(max_depth=i)
    tree.fit(train_data[features], train_data[target])
    dt_accuracies.append((
        accuracy_score(tree.predict(train_data[features]), train_data[target]),
        accuracy_score(tree.predict(validation_data[features]), validation_data[target])
    ))
    
    # Train and evaluate our RandomForest classifier with given max_depth 
    rf = RandomForest416(15, max_depth=i)
    rf.fit(train_data[features], train_data[target])
    rf_accuracies.append((     
        accuracy_score(rf.predict(train_data[features]), train_data[target]),
        accuracy_score(rf.predict(validation_data[features]), validation_data[target])
    ))
    
# Then plot the scores
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Plot training accuracies
axs[0].plot(depths, [acc[0] for acc in dt_accuracies], label='DecisionTree')
axs[0].plot(depths, [acc[0] for acc in rf_accuracies], label='RandomForest416')

# Plot validation accuracies
axs[1].plot(depths, [acc[1] for acc in dt_accuracies], label='DecisionTree')
axs[1].plot(depths, [acc[1] for acc in rf_accuracies], label='RandomForest416')

# Customize plots
axs[0].set_title('Train Data')
axs[1].set_title('Validation Data')
for ax in axs:
    ax.legend()
    ax.set_xlabel('Max Depth')
    ax.set_ylabel('Accuracy')